## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN','NAME' and others.
application_df = application_df.drop(columns= ["EIN", "NAME", "AFFILIATION", "ORGANIZATION", "SPECIAL_CONSIDERATIONS"])

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
CLASSIFICATION        71
USE_CASE               5
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [4]:
# Look at ASK_AMT value counts for binning (the number of data points for each unique value)
ask_amount = application_df["ASK_AMT"].value_counts()
ask_amount

5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df["APPLICATION_TYPE"].value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_counts[app_counts < 300].index)
application_types_to_replace

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [7]:
# Replace in dataframe
for app in application_types_to_replace:
      application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df["CLASSIFICATION"].value_counts()
class_counts.head(40)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_counts[class_counts < 100].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [10]:
# Separate the features, X, from the target variable, y
X = application_df.drop(columns = "IS_SUCCESSFUL")
y = application_df["IS_SUCCESSFUL"]

In [11]:
# Preview the features data
X.head()

,APPLICATION_TYPE,CLASSIFICATION,USE_CASE,STATUS,INCOME_AMT,ASK_AMT
0,T10,C1000,ProductDev,1,0,5000
1,T3,C2000,Preservation,1,1-9999,108590
2,T5,C3000,ProductDev,1,0,5000
3,T3,C2000,Preservation,1,10000-24999,6692
4,T3,C1000,Heathcare,1,100000-499999,142590


In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
X = pd.get_dummies(X)
X.head()

,STATUS,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,USE_CASE_ProductDev,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,0,1,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
1,1,108590,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,1,0,0,...,1,1,0,0,0,0,0,0,0,0
3,1,6692,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,142590,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [13]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

In [14]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
X_train_scaled

array([[ 0.01079982, -0.03243656, -0.08940766, ...,  2.86649057,
        -0.06340457, -0.07450354],
       [ 0.01079982, -0.03253341, -0.08940766, ..., -0.34885864,
        -0.06340457, -0.07450354],
       [ 0.01079982, -0.03241136, -0.08940766, ..., -0.34885864,
        -0.06340457, -0.07450354],
       ...,
       [ 0.01079982, -0.03149733, -0.08940766, ..., -0.34885864,
        -0.06340457, -0.07450354],
       [ 0.01079982, -0.03253341, -0.08940766, ..., -0.34885864,
        -0.06340457, -0.07450354],
       [ 0.01079982, -0.03253341, -0.08940766, ..., -0.34885864,
        -0.06340457, -0.07450354]])

## Compile, Train and Evaluate the Model

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.iloc[0])
hidden_layer1 = 16
hidden_layer2 = 12


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_layer1, input_dim = number_input_features, activation = "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_layer2, activation = "LeakyReLU"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                608       
                                                                 
 dense_1 (Dense)             (None, 12)                204       
                                                                 
 dense_2 (Dense)             (None, 1)                 13        
                                                                 
Total params: 825 (3.22 KB)
Trainable params: 825 (3.22 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Define the path to save the weights
checkpoint_path = 'model_weights.h5'

# Create a ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    checkpoint_path,
    save_weights_only=True,  # Only save the weights, not the entire model
    save_freq=5,      # Save weights every 5 epochs
)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=110)


Epoch 1/110
804/804 [==============================] - 2s 2ms/step - loss: 0.6202 - accuracy: 0.6338
Epoch 2/110
804/804 [==============================] - 1s 2ms/step - loss: 0.6203 - accuracy: 0.6356
Epoch 3/110
804/804 [==============================] - 2s 2ms/step - loss: 0.6202 - accuracy: 0.6341
Epoch 4/110
804/804 [==============================] - 2s 3ms/step - loss: 0.6204 - accuracy: 0.6313
Epoch 5/110
804/804 [==============================] - 2s 2ms/step - loss: 0.6205 - accuracy: 0.6343
Epoch 6/110
804/804 [==============================] - 1s 2ms/step - loss: 0.6207 - accuracy: 0.6331
Epoch 7/110
804/804 [==============================] - 2s 2ms/step - loss: 0.6204 - accuracy: 0.6332
Epoch 8/110
804/804 [==============================] - 2s 2ms/step - loss: 0.6204 - accuracy: 0.6336
Epoch 9/110
804/804 [==============================] - 2s 2ms/step - loss: 0.6201 - accuracy: 0.6327
Epoch 10/110
804/804 [==============================] - 2s 2ms/step - loss: 0.6202 - accura

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6291 - accuracy: 0.6269 - 327ms/epoch - 1ms/step
Loss: 0.6290655732154846, Accuracy: 0.6269387602806091
